In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk


In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
df = pd.read_csv('/content/Amazon Fashion New.csv')

In [ ]:
df.drop(columns=['asin', 'reviewTime', 'reviewerID', 'reviewerName', 'unixReviewTime', 'image', 'style', 'vote'], inplace=True)

In [ ]:
print(df.head())

                                          reviewText  overall  \
0                             Exactly what I needed.        5   
1  I agree with the other review, the opening is ...        2   
2  Love these... I am going to order another pack...        4   
3                                too tiny an opening        2   
4                                               Okay        3   

                                             summary  verified  
0                             perfect replacements!!      True  
1  I agree with the other review, the opening is ...      True  
2                                My New 'Friends' !!     False  
3                                          Two Stars      True  
4                                        Three Stars     False  


In [ ]:
print("\nMissing values:")
print(df.isnull().sum())


Missing values:
reviewText    14
overall        0
summary        4
verified       0
dtype: int64


In [ ]:
df['summary'].fillna('', inplace=True)
df['reviewText'].fillna('', inplace=True)

In [ ]:
print(df.isnull().sum())

reviewText    0
overall       0
summary       0
verified      0
dtype: int64


In [ ]:
df['text'] = df['reviewText'] + ' ' + df['summary']

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def additional_clean(text):
    text = str(text)
    text = re.sub(r'http\S+', '', text)
    # Convert contractions
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(filtered_tokens)
    # Lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    # Spell correction
    text = str(TextBlob(text).correct())

    specific_words = ['buxton', 'birkenstock', 'one', 'week', 'wanted', 'postage', 'clergy', 'use', 'piece', 'hour', 'calvin', 'ear','looking', 'still','cigarette', 'used', 'arrived', 'much', 'thing', 'hold', 'cotton', 'find', 'husband', 'skin', 'recieved', 'silver', 'back', 'probably', 'case', 'made', 'metal', 'worn', 'finally', 'five','large', 'donate', 'friend','boyfriend', 'v', 'grm', 'star', 'im', 'amazon', 'year', 'old', 'foot', 'way', 'four', 'right', 'earning', 'first', 'shipping' 'one', 'time', 'picture','shirt', 'wear', 'Really', 'material', 'think', 'even', 'make', 'two', 'three', 'month']
    text = ' '.join([word for word in text.split() if word not in specific_words])
    return text

In [ ]:
df['additional_clean'] = df['text'].apply(additional_clean)

In [ ]:
df.to_csv('preprocessed_data.csv', index=False)

In [ ]:
for index, row in df.iterrows():
    print(row['additional_clean'])

Streaming output truncated to the last 5000 lines.
item stated good quality however size shipped correct tried contact vendor email phone exchange return purchase received response stuck poor service
washed nicely fit well problem size neck needed neck ordered perfect promo
ordered wash well pleased good value money hard get well
look nice fit well excellent buy twice department store tax
great quality slack love fit length sturdiness etc bad carhartt changed front leg velcroed pocket full cut velcro across top easily slip mobile pad pocket weird slantedcut velcro thicker slip mobile etc maybe resistant picked preferred older style terrific slack favorite love fit like new pocket
happy comfort practically part happy wash well look good nice part
brand jumping settled carhartt everything regret live colder climate winter outdoor gear stuff super well last long farm carhartt great stuff
know size getting via great part unable fit great good work part
size x levi carhartts fit perfectly p

In [ ]:
def map_sentiment_label(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

In [ ]:
df['sentiment'] = df['overall'].apply(map_sentiment_label)

In [ ]:
sentiment_counts = df['sentiment'].value_counts(normalize=True)
print("Sentiment Label Distribution:")
print(sentiment_counts)

Sentiment Label Distribution:
sentiment
positive    0.7862
negative    0.1159
neutral     0.0979
Name: proportion, dtype: float64
